In [ ]:
import os, subprocess

In [ ]:
yolov4_base_path = "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415"

In [ ]:
os.makedirs(yolov4_base_path, exist_ok=True)

# darknet 설치
https://webnautes.tistory.com/1482

In [ ]:
os.chdir(yolov4_base_path)

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Makefile
```
GPU=1 
CUDNN=1 
CUDNN_HALF=0 
OPENCV=1 
AVX=0 
OPENMP=0 
LIBSO=1
```

In [ ]:
darknet_path = f"{yolov4_base_path}/darknet"
print(darknet_path)

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
!make

# darknet 파일 추가, 수정하기

폴더 구조
- darknet
  - data folder
    - labels folder
    - obj folder
    - obj.data file
    - obj.names file
    - (test.txt)
    - (train.txt)
  - cfg folder
    - yolov4-custom.cfg
- training

## cfg 폴더의 yolov4-custom.cfg
1. yolov4-custom.cfg 외 모든 파일 삭제
2. yolov4-custom.cfg 수정
   - batch=64
   - **subdivision=16 or 32 or 64**
   - width=416, height=416
   - max_batches = (class 개수) * 2000 (단, training images 수 이상, 6000 이상)
   - steps=80% and 90% of max_batches
   - [yolo] 직전에 있는, 3개의 [convolutional] filters=(classes + 5) * 3
   - 3개의 [yolo] classes = (class 개수)
   - **learning rate, angle, saturation, exposure, hue 추가로 바꿀 수 있음**

yolov4-custom.cfg 수정
   - batch=64
   - **subdivision=16 or 32 or 64** 
     - 32 설정
   - width=416, height=416
   - max_batches = 12000
     - ArUcoMarkers(11541)
   - steps = 9600,10800
   - [yolo] 직전에 있는, 3개의 [convolutional] filters=30
   - 3개의 [yolo] classes = 5
   - **learning rate, angle, saturation, exposure, hue 추가로 바꿀 수 있음**

## data 폴더
1. labels 폴더 외 모두 삭제
2. obj.data
3. obj.names

## obj.data

In [ ]:
# 클래스 인데싱 딕셔너리
class_dir = {'0': 0, '10': 1, '20': 2, '30': 3, '40': 4} # 수정, 추가 필요

In [ ]:
with open(darknet_path+"/data/obj.data", "w") as f:
    lines = f"classes = {len(class_dir)}\n"
    lines += f"train = {darknet_path}/data/train.txt\n"
    lines += f"valid = {darknet_path}/data/test.txt\n"
    lines += f"names = {darknet_path}/data/obj.names\n"
    lines += f"backup = {yolov4_base_path}/training\n"
    f.write(lines)

## obj.names

In [ ]:
with open(darknet_path+"/data/obj.names", "w") as f:
    lines = ""
    for name in class_dir.keys():
        lines += name + '\n'
    f.write(lines)

## 학습 결과 weights 파일 저장될 Training 폴더 만들기

In [ ]:
os.makedirs(yolov4_base_path+"/training", exist_ok=True)

---

# Labeled Custom Data

1. 데이터셋 파일 정리하기
2. 데이터셋 파일 리스트 만들기

## train.txt & test.txt 만들기
shuffle 한 후 train test split

# download weights

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
download_weight = "wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"
subprocess.call(list(download_weight.split()))

# Training

In [ ]:
print(darknet_path)

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
!./darknet detector train "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/yolov4.conv.137" -map

# Check mAP

In [ ]:
!./darknet detector map "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/training/yolov4-custom_best.weights" -points 0

# Run detector on a live webcam

In [ ]:
!./darknet detector demo "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/data/obj.data" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/darknet/cfg/yolov4-custom.cfg" "/home/erbl/Git/RealTime-Object-Detection/yolov4-220415/training/yolov4-custom_best.weights" -thresh 0.5